## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
import warnings
warnings.filterwarnings('ignore')

#matplotlib inline
import matplotlib.pylab as plt
## for compute y 
from data_utils import TextMelLoader, TextMelCollate
from torch.utils.data import DataLoader




import IPython.display as ipd
import os 
#os.chdir('/research/crissp/lebbat/NVIDIA-official/tacotron2/')
print(os.getcwd())
import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from waveglow.denoiser import Denoiser
import random

#random.seed(0)
#np.random.seed(0)
os.getcwd()
#torch.manual_seed(0)



/research/crissp/lebbat/NVIDIA-official/tacotron2


'/research/crissp/lebbat/NVIDIA-official/tacotron2'

In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

In [3]:
def plot_gate_outputs_to_numpy(gate_outputs):
    fig, ax = plt.subplots(figsize=(12, 3))
    #ax.scatter(range(len(gate_targets)), gate_targets, alpha=0.7,
               #color='green', marker='+', s=3, label='target')
    ax.scatter(range(len(gate_outputs)), gate_outputs, alpha=1,
               color='red', marker='.', s=10, label='predicted')

    plt.xlabel("Frames ( Red predicted)")
    plt.ylabel("Gate State")
    plt.tight_layout()

    fig.canvas.draw()
    plt.show()
    plt.close()


#### Setup hparams

In [4]:
hparams = create_hparams()
hparams.sampling_rate = 16000

#### Load model from checkpoint

In [5]:
checkpoint_path = "/research/crissp/lebbat/NVIDIA-official/tacotron2/outdir/checkpoint_32000"
#checkpoint_path = "tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

holy cow


#### Load WaveGlow for mel2audio synthesis and denoiser

In [6]:
waveglow_path = 'waveglow_256channels_ljs_v2_new.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [7]:
def prepare_dataloaders(hparams):
    trainset = TextMelLoader(hparams.training_files, hparams)
    valset = TextMelLoader(hparams.validation_files, hparams)
    collate_fn = TextMelCollate(hparams.n_frames_per_step)

    train_loader = DataLoader(trainset, num_workers=1, shuffle=False,
                              sampler=None,
                              batch_size=hparams.batch_size, pin_memory=False,
                              drop_last=False, collate_fn=collate_fn)
    return train_loader, valset, collate_fn

In [8]:
train_loader, valset, collate_fn = prepare_dataloaders(hparams)
for i, batch in enumerate(train_loader):
    x, y = model.parse_batch(batch)
    y_pred = model(x)
    #print(batch)
    #print(y)
    #print(x)

tensor([126, 125], device='cuda:0') This is output_lengths.data at forward


RuntimeError: Expected object of scalar type Half but got scalar type Float for sequence element 1 in sequence argument at position #1 'tensors'

#### Prepare text input

In [ ]:
text = "—Me direz-vous quels sont les parents de cette jeune fille dont l'enfance a été torturée par la Chouette?"
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [ ]:
text2 = ".je m'appelle Taoufik."
sequence2 = np.array(text_to_sequence(text2, ['basic_cleaners']))[None, :]
sequence2 = torch.autograd.Variable(
    torch.from_numpy(sequence2)).cuda().long()

In [ ]:
text3 = ";je m'appelle Taoufik;"
sequence3 = np.array(text_to_sequence(text3, ['basic_cleaners']))[None, :]
sequence3 = torch.autograd.Variable(
    torch.from_numpy(sequence3)).cuda().long()

In [ ]:
text4 = "-Bonjour Tacotron ..."
sequence4 = np.array(text_to_sequence(text4, ['basic_cleaners']))[None, :]
sequence4 = torch.autograd.Variable(
    torch.from_numpy(sequence4)).cuda().long()

#### Decode text input and plot results

In [ ]:
mel_outputs, mel_outputs_postnet, gate_outputs, alignments = model.inference(sequence,hparams)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))
idx = random.randint(0, alignments.size(0) - 1)
plot_gate_outputs_to_numpy(
                torch.sigmoid(gate_outputs[idx]).data.cpu().numpy()) # ,dataformats='HWC'

In [ ]:
mel_outputs2, mel_outputs_postnet2, _, alignments2 = model.inference(sequence2,hparams)
plot_data((mel_outputs2.float().data.cpu().numpy()[0],
           mel_outputs_postnet2.float().data.cpu().numpy()[0],
           alignments2.float().data.cpu().numpy()[0].T))
idx = random.randint(0, alignments.size(0) - 1)
plot_gate_outputs_to_numpy(
                torch.sigmoid(gate_outputs[idx]).data.cpu().numpy()) # ,dataformats='HWC'


In [ ]:
mel_outputs3, mel_outputs_postnet3, _, alignments3 = model.inference(sequence3,hparams)
plot_data((mel_outputs3.float().data.cpu().numpy()[0],
           mel_outputs_postnet3.float().data.cpu().numpy()[0],
           alignments3.float().data.cpu().numpy()[0].T))

idx = random.randint(0, alignments.size(0) - 1)
plot_gate_outputs_to_numpy(
                torch.sigmoid(gate_outputs[idx]).data.cpu().numpy()) # ,dataformats='HWC'

In [ ]:
mel_outputs4, mel_outputs_postnet4, _, alignments4 = model.inference(sequence4,hparams)
plot_data((mel_outputs4.float().data.cpu().numpy()[0],
           mel_outputs_postnet4.float().data.cpu().numpy()[0],
           alignments4.float().data.cpu().numpy()[0].T))
idx = random.randint(0, alignments.size(0) - 1)
plot_gate_outputs_to_numpy(
                torch.sigmoid(gate_outputs[idx]).data.cpu().numpy()) # ,dataformats='HWC'

#### Synthesize audio from spectrogram using WaveGlow

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.6)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
with torch.no_grad():
    audio2 = waveglow.infer(mel_outputs_postnet2, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
with torch.no_grad():
    audio3 = waveglow.infer(mel_outputs_postnet3, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet4, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [ ]:
audio_denoised = denoiser(audio, strength=0.03)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

In [ ]:
audio_denoised = denoiser(audio2, strength=0.02)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

In [ ]:
audio_denoised = denoiser(audio3, strength=0.02)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 